In [1]:
file_path="/home/jovyan/work/persons.xml"

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("data-gurus").getOrCreate()
print(spark)

In [4]:
tree = ET.parse(file_path)
root = tree.getroot()
print(root)

<Element 'persons' at 0x7f0ea49f4b30>


In [5]:
###FIRST METHOD 
### ONE DICT WITH KEYS AND VALUES IN LISTS
data=[]
my_dict={}
for elem in root:
    for sub_elem in elem:
        # check if key already exists in the dictionary
        if my_dict.__contains__(sub_elem.tag):
            try:
                my_dict[sub_elem.tag].append(sub_elem.text)
                print("sub elem: ",sub_elem.text ,type(sub_elem.text))
            except:
                continue
        # else create the new key and append
        else:
            try:
                my_dict[sub_elem.tag] = []
                my_dict[sub_elem.tag].append(sub_elem.text)
                print("sub elem: ",sub_elem.text ,type(sub_elem.text))
            except:
                continue
    #data.append(my_dict)

sub elem:  James <class 'str'>
sub elem:  Smith <class 'str'>
sub elem:  None <class 'NoneType'>
sub elem:  1980 <class 'str'>
sub elem:  1 <class 'str'>
sub elem:  M <class 'str'>
sub elem:  10000 <class 'str'>
sub elem:  
             <class 'str'>
sub elem:  Michael <class 'str'>
sub elem:  None <class 'NoneType'>
sub elem:  Rose <class 'str'>
sub elem:  1990 <class 'str'>
sub elem:  6 <class 'str'>
sub elem:  M <class 'str'>
sub elem:  10000 <class 'str'>
sub elem:  
             <class 'str'>


In [6]:
print(my_dict)

{'firstname': ['James', 'Michael'], 'lastname': ['Smith', None], 'middlename': [None, 'Rose'], 'dob_year': ['1980', '1990'], 'dob_month': ['1', '6'], 'gender': ['M', 'M'], 'salary': ['10000', '10000'], 'addresses': ['\n            ', '\n            ']}


In [7]:
## SECONG MENTHOD
## LIST WITH SEPARATE DICTS 
#data list
data=[]
for elem in root:
    my_dict={}
    for sub_elem in elem:
        try:
            my_dict[sub_elem.tag]=(sub_elem.text)
            print("sub elem: ",sub_elem.text ,type(sub_elem.text))
        except:
            continue
    data.append(my_dict)

sub elem:  James <class 'str'>
sub elem:  Smith <class 'str'>
sub elem:  None <class 'NoneType'>
sub elem:  1980 <class 'str'>
sub elem:  1 <class 'str'>
sub elem:  M <class 'str'>
sub elem:  10000 <class 'str'>
sub elem:  
             <class 'str'>
sub elem:  Michael <class 'str'>
sub elem:  None <class 'NoneType'>
sub elem:  Rose <class 'str'>
sub elem:  1990 <class 'str'>
sub elem:  6 <class 'str'>
sub elem:  M <class 'str'>
sub elem:  10000 <class 'str'>
sub elem:  
             <class 'str'>


In [8]:
#list of dict
print(data)

[{'firstname': 'James', 'lastname': 'Smith', 'middlename': None, 'dob_year': '1980', 'dob_month': '1', 'gender': 'M', 'salary': '10000', 'addresses': '\n            '}, {'firstname': 'Michael', 'lastname': None, 'middlename': 'Rose', 'dob_year': '1990', 'dob_month': '6', 'gender': 'M', 'salary': '10000', 'addresses': '\n            '}]


In [9]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType, IntegerType

In [10]:
#schema dictionary
schemaDict = {'persons': StructType([StructField('firstname', StringType(), True),
                                 StructField('lastname', StringType(), True),
                                 StructField('middlename', StringType(), True),
                                 StructField('dob_year', StringType(), True),
                                 StructField('dob_month', StringType(), True),
                                 StructField('gender', StringType(), True),
                                 StructField('salary', StringType(), True),
                                StructField('addresses', StringType(), True)
                                ])}

In [ ]:
sdf = spark.createDataFrame(data, schema=schemaDict['persons'])
sdf.printSchema()

In [ ]:
sdf.show()

In [ ]:
table_name=file_name
sdf.write.option("path", f"file:/home/jovyan/spark-warehouse/marktstammdaten_{file_name.lower()}").mode("append").saveAsTable(table_name)
          

In [ ]:
aes = spark.sql("select * from persons")
print(aes.count())
print(aes.schema)


In [ ]:
aes.show()

In [41]:
#Example of literal_eval
try:
    #string
    a = '9082'
    #int conversion
    b = ast.literal_eval(a)
except SyntaxError:
    b = ast.literal_eval(a)
print(type(b))

NameError: name 'ast' is not defined

In [54]:
#TRYING PANDAS


xml_data = open(file_path, 'r').read()  # Read file
root = ET.XML(xml_data)  # Parse XML

data = []
cols = []
for i, child in enumerate(root):
    data.append([subchild.text for subchild in child])

cols.append([subchild.tag for subchild in child])

print(data)
df = pd.DataFrame(data)  # Write in DF and transpose it
df.columns = cols  # Update column names
print(df)

[['James', 'Smith', None, '1980', '1', 'M', '10000', '\n            '], ['Michael', None, 'Rose', '1990', '6', 'M', '10000', '\n            ']]
  firstname lastname middlename dob_year dob_month gender salary  \
0     James    Smith       None     1980         1      M  10000   
1   Michael     None       Rose     1990         6      M  10000   

        addresses  
0  \n              
1  \n              


In [43]:
df.dtypes

firstname     object
lastname      object
middlename    object
dob_year      object
dob_month     object
gender        object
salary        object
addresses     object
dtype: object

In [44]:
#checking schema
df.info(verbose = True, show_counts = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   (firstname,)   object
 1   (lastname,)    object
 2   (middlename,)  object
 3   (dob_year,)    object
 4   (dob_month,)   object
 5   (gender,)      object
 6   (salary,)      object
 7   (addresses,)   object
dtypes: object(8)
memory usage: 256.0+ bytes


In [45]:
#schema dictionary
schemaDict = {'persons': StructType([StructField('firstname', StringType(), True),
                                 StructField('lastname', StringType(), True),
                                 StructField('middlename', StringType(), True),
                                 StructField('dob_year', LongType(), True),
                                 StructField('dob_month', IntegerType(), True),
                                 StructField('gender', StringType(), True),
                                 StructField('salary', LongType(), True),
                                StructField('addresses', StringType(), True)
                                ])}

In [46]:
sdf = spark.createDataFrame(df, schema=schemaDict['persons'])
sdf.printSchema()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


TypeError: field dob_year: LongType() can not accept object '1980' in type <class 'str'>

In [55]:
new_data=[['James', 'Smith', None, 1980, 1, 'M', 10000, '\n            '], ['Michael', None, 'Rose', 1990, 6, 'M', 10000, '\n            ']]

In [71]:
new_data = [[10000001,1,0,1,'12:35','OK',10002,1,0,9,'f','NA',24,24,0,3,9,0,0,1,1,0,0,4,543],
[10000001,2,0,1,'12:36','OK',10002,1,0,9,'f','NA',24,24,0,3,9,2,1,1,3,1,3,2,611],
[10000002,1,0,4,'12:19','PA',10003,1,1,7,'f','NA',74,74,0,2,15,2,0,2,3,1,2,2,691]]

In [76]:
new_df=pd.DataFrame(new_data)
print(type(new_df))
cols=["num 1","num 2","num 3","num 4","time","status","num 5","num 6","num 7","num 8","f", "status", "num","num","num","num","num","num","num","num","num","num","num","num","num",]
new_df.columns=cols
print(new_df.head)


<class 'pandas.core.frame.DataFrame'>
<bound method NDFrame.head of       num 1  num 2  num 3  num 4   time status  num 5  num 6  num 7  num 8  \
0  10000001      1      0      1  12:35     OK  10002      1      0      9   
1  10000001      2      0      1  12:36     OK  10002      1      0      9   
2  10000002      1      0      4  12:19     PA  10003      1      1      7   

   ...  num  num  num  num  num  num  num  num  num  num  
0  ...    3    9    0    0    1    1    0    0    4  543  
1  ...    3    9    2    1    1    3    1    3    2  611  
2  ...    2   15    2    0    2    3    1    2    2  691  

[3 rows x 25 columns]>


In [78]:
from pyspark.sql import SQLContext
sc=spark.sparkContext
sqlContext = SQLContext(sc)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [79]:
##NEW TESTING PANDAS TO PYSPARK

from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return DoubleType()
    elif f == 'float32': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

spark_df = pandas_to_spark(new_df)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


ValueError: name already used as a name or title